In [7]:
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def neighborhood_lower_bound(graph):
    nodes = list(graph.nodes())
    n = len(nodes)
    lower_bounds = {}

    k = 2
    max_iterations = 1000  # 设置最大迭代次数

    Y = {}
    S_un = {}
    nVisited = {}
    finished = {}

    for s in nodes:
        degree_s = graph.degree(s)
        Y[(k - 1, s)] = degree_s
        S_un[(k - 1, s)] = degree_s
        nVisited[s] = degree_s + 1
        finished[s] = False

    nFinished = 0
    while nFinished < n and k <= max_iterations:
        for s in nodes:
            if finished[s]:
                continue

            if k == 2:
                Y[(k, s)] = sum(Y[(k - 1, w)] for w in graph.neighbors(s)) - graph.degree(s)
            elif k > 2:
                Y[(k, s)] = sum(Y[(k - 1, w)] for w in graph.neighbors(s)) - Y.get((k - 2, s), 0) * (graph.degree(s) - 1)
            else:
                Y[(k, s)] = 0

            nVisited[s] += Y.get((k - 1, s), 0)

            S_un[(k, s)] = S_un.get((k - 1, s), 0) + k * Y.get((k - 1, s), 0)

            if nVisited[s] >= n:
                finished[s] = True
                nFinished += 1

        k += 1

    for v in nodes:
        lower_bounds[v] = S_un.get((k - 1, v), 0) / (n-1)

    return lower_bounds


def updateBoundsLB(graph, s):
    """
    计算无向图中割集大小的下界。

    参数：
        s: 源节点。

    返回：
        一个字典，键是节点，值是它们的下界割集大小。
    """

    nodes = list(graph.nodes())
    n = len(nodes)
    d = nx.single_source_shortest_path_length(graph, s)
    maxD = max(d.values())
    sumL = {i:0 for i in range(maxD+2)}
    sumG = {i:0 for i in range(maxD+2)}

    Gamma = {i: [] for i in range(maxD + 1)}
    for v, dist in d.items():
        Gamma[dist].append(v)

    for i in range(maxD+1):
        sumL[i+1] = sumL[i] + len(Gamma[i]) #这是伪代码中的yi


    sumG[maxD+1] = len(graph.nodes)
    for i in range(maxD,0,-1):
        sumG[i] = n - sumL[i+1]

    L = {}
    L[1] = sumL[1] + sumL[2] + sumG[2] -2


    for i in range(2, maxD+1):
        L[i] = L[i-1] + sumL[i-2] - sumG[i+1]

    LB_s = {}
    for i in range(1, maxD+1):
        for v in Gamma[i]:
            #LB 计算，注意 r(v) 应该定义为图中每个节点的属性
            LB_s[v] = L[i] - graph.degree(v) / (len(graph.nodes)-1) 


    return LB_s
    

def create_mention_graph_with_centrality(filepath, top_k=10):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds...")
    lower_bounds = neighborhood_lower_bound(graph)
    nx.set_node_attributes(graph, lower_bounds, 'lower_centrality')

    # 获取并打印 top k 个节点（基于初始下界）
    sorted_centralities = sorted(
        lower_bounds.items(),
        key=lambda x: x[1],
        reverse=True
    )[:top_k]

    print(f"\nTop {top_k} nodes by initial lower centrality bound:")
    for i, (node, centrality) in enumerate(sorted_centralities, 1):
        print(f"{i}. Node: {node}, Initial Lower Centrality: {centrality:.10f}")

    print(f"\nRefining bounds for top {top_k} nodes...")
    for node, _ in sorted_centralities:
        refined_bounds = updateBoundsLB(graph, node)
        nx.set_node_attributes(graph, {v: {'refined_lower_centrality': b} for v, b in refined_bounds.items()})

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    # 获取并打印精炼后的top 10结果
    refined_centralities = nx.get_node_attributes(graph, 'refined_lower_centrality')
    sorted_refined_centralities = sorted(
        refined_centralities.items(),
        key=lambda x: x[1],
        reverse=True
    )[:10]

    print("\nTop 10 nodes by refined lower centrality bound:")
    for i, (node, centrality) in enumerate(sorted_refined_centralities, 1):
        print(f"{i}. Node: {node}, Refined Lower Centrality: {centrality:.10f}")

    return graph

if __name__ == "__main__":
    filepath = "medium.tsv"
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

Creating graph...
Undirected graph created with 2158 nodes and 5136 edges.
Calculating initial lower bounds...

Top 10 nodes by initial lower centrality bound:
1. Node: 1849, Initial Lower Centrality: 255.5108947612
2. Node: 1811, Initial Lower Centrality: 214.2749188688
3. Node: 1813, Initial Lower Centrality: 214.2749188688
4. Node: 1814, Initial Lower Centrality: 214.2749188688
5. Node: 1815, Initial Lower Centrality: 214.2749188688
6. Node: 1816, Initial Lower Centrality: 214.2749188688
7. Node: 1817, Initial Lower Centrality: 214.2749188688
8. Node: 1820, Initial Lower Centrality: 214.2749188688
9. Node: 1821, Initial Lower Centrality: 214.2749188688
10. Node: 1822, Initial Lower Centrality: 214.2749188688

Refining bounds for top 10 nodes...

Total time taken: 0.0789 seconds

Top 10 nodes by refined lower centrality bound:
1. Node: 689, Refined Lower Centrality: 2036.9439035698
2. Node: 526, Refined Lower Centrality: 728.9995363931
3. Node: 562, Refined Lower Centrality: 728.9995

In [31]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def neighborhood_lower_bound(graph):
    nodes = list(graph.nodes())
    n = len(nodes)
    lower_bounds = {}

    k = 2
    max_iterations = 1000  # 设置最大迭代次数

    Y = {}
    S_un = {}
    nVisited = {}
    finished = {}

    for s in nodes:
        degree_s = graph.degree(s)
        Y[(k - 1, s)] = degree_s
        S_un[s] = degree_s
        nVisited[s] = degree_s + 1
        finished[s] = False

    nFinished = 0
    while nFinished < n and k <= max_iterations:
        for s in nodes:
            if finished[s]:
                continue

            if k == 2:
                Y[(k, s)] = sum(Y[(k - 1, w)] for w in graph.neighbors(s)) - graph.degree(s)
            elif k > 2:
                Y[(k, s)] = sum(Y[(k - 1, w)] for w in graph.neighbors(s)) - Y.get((k - 2, s), 0) * (graph.degree(s) - 1)
            else:
                Y[(k, s)] = 0

            nVisited[s] += Y.get((k - 1, s), 0)

            S_un[s] = S_un[s] + k * Y.get((k - 1, s), 0)

            if nVisited[s] >= n:
                finished[s] = True
                nFinished += 1

        k += 1

    for v in nodes:
        lower_bounds[v] = S_un[v]/ (n-1)

    return lower_bounds


def updateBoundsLB(graph, s):
    nodes = list(graph.nodes())
    n = len(nodes)
    d = nx.single_source_shortest_path_length(graph, s)
    maxD = max(d.values())
    sumL = {i: 0 for i in range(maxD+2)}
    sumG = {i: 0 for i in range(maxD+2)}

    Gamma = {i: [] for i in range(maxD + 1)}
    for v, dist in d.items():
        Gamma[dist].append(v)
    sumG[maxD+1] = 0
    for i in range(maxD):
        sumL[i+1] = sumL[i] + len(Gamma[i+1])
        sumG[i+1] = n - sumL[i+1]

    L = {}
    L[1] = len(Gamma[1]) + len(Gamma[2]) + sumG[2] - 2

    for i in range(2, maxD+1):
        L[i] = L[i-1] + sumL[i-2] - sumG[i+1]

    LB_s = {}
    for i in range(1, maxD+1):
        for v in Gamma[i]:
            LB_s[v] = (L[i] - graph.degree(v))/ (n-1)

    return LB_s

def create_mention_graph_with_centrality(filepath, top_k=20):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")

    # 算法 1 实现
    L = neighborhood_lower_bound(graph)

    k_lowest_farness = heapq.nsmallest(top_k, L.items(), key=lambda item: item[1])
    print(f"\n{top_k} nodes with lowest farness values:")
    for node, farness in k_lowest_farness:
        print(f"Node: {node}, Farness: {farness}")

    negative_farness_nodes = {node: farness for node, farness in L.items() if farness < 0}
    if negative_farness_nodes:
        print("\nNodes with negative farness values:")
        for node, farness in negative_farness_nodes.items():
            print(f"Node: {node}, Farness: {farness}")
    else:
        print("\nNo nodes with negative farness values found.")
        
    Q = [(L[v], v) for v in graph.nodes()]
    heapq.heapify(Q)
    Top = []
    Farm = L.copy()

    _, v = heapq.heappop(Q)
    while Q and len(Top) < top_k:
        #
        if len(Top) >= top_k and Farm[v] > Farm[Top[-1]]:
            break
        
        Top.append(v)
        Top.sort(key=lambda x: Farm[x], reverse=True)

        negative_farness_nodes_after_update = {node: farness for node, farness in Farm.items() if farness < 0}
        if negative_farness_nodes_after_update:
            print("\nNodes with negative farness values (after update):")
            for node, farness in negative_farness_nodes_after_update.items():
                print(f"Node: {node}, Farness: {farness}")
        else:
            print("\nNo nodes with negative farness values found (after update).")

        # 更新 Q：更新 Farm 后重新堆化
        Q = [(Farm[u], u) for u in graph.nodes() if u not in Top]
        heapq.heapify(Q)
        v_1 = v 
        _, v = heapq.heappop(Q)
        refined_bounds = updateBoundsLB(graph, v_1)
        Farm[v] = refined_bounds[v]

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")
    
    print(f"\nTop {top_k} nodes:")
    Cen = {}
    for i, node in enumerate(Top[:top_k], 1):
        #Cen[node] = 1/Farm[node]
        #print(f"{i}. Node: {node}, Refined Lower Centrality: {Cen[node]:.10f}")
        print(f"{i}. Node: {node}, Refined Lower Centrality: {Farm[node]:.10f}")

    return graph

if __name__ == "__main__":
    filepath = "medium.tsv"  # 替换为你文件的路径
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

Creating graph...
Undirected graph created with 2158 nodes and 5136 edges.

20 nodes with lowest farness values:
Node: 575, Farness: 3.001390820584145
Node: 374, Farness: 3.008344923504868
Node: 851, Farness: 3.0139082058414464
Node: 222, Farness: 3.044506258692629
Node: 46, Farness: 3.097357440890125
Node: 588, Farness: 3.09874826147427
Node: 682, Farness: 3.2962447844228095
Node: 628, Farness: 3.305980528511822
Node: 529, Farness: 3.3310152990264257
Node: 245, Farness: 3.388038942976356
Node: 817, Farness: 3.4089012517385258
Node: 770, Farness: 3.4255910987482614
Node: 118, Farness: 3.49095966620306
Node: 116, Farness: 3.4937413073713492
Node: 717, Farness: 3.5076495132127956
Node: 51, Farness: 3.5424200278164117
Node: 22, Farness: 3.556328233657858
Node: 572, Farness: 3.603616133518776
Node: 270, Farness: 3.631432545201669
Node: 611, Farness: 3.6481223922114046

No nodes with negative farness values found.

No nodes with negative farness values found (after update).

No nodes with n

In [17]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def neighborhood_lower_bound(graph):
    nodes = list(graph.nodes())
    n = len(nodes)
    lower_bounds = {}

    k = 2
    max_iterations = 1000  # 设置最大迭代次数

    Y = {}
    S_un = {}
    nVisited = {}
    finished = {}

    for s in nodes:
        degree_s = graph.degree(s)
        Y[(k - 1, s)] = degree_s
        S_un[s] = degree_s
        nVisited[s] = degree_s + 1
        finished[s] = False

    nFinished = 0
    while nFinished < n and k <= max_iterations:
        for s in nodes:
            if finished[s]:
                continue

            if k == 2:
                Y[(k, s)] = sum(Y[(k - 1, w)] for w in graph.neighbors(s)) - graph.degree(s)
            elif k > 2:
                Y[(k, s)] = sum(Y[(k - 1, w)] for w in graph.neighbors(s)) - Y.get((k - 2, s), 0) * (graph.degree(s) - 1)
            else:
                Y[(k, s)] = 0

            nVisited[s] += Y.get((k - 1, s), 0)

            S_un[s] = S_un[s] + k * Y.get((k - 1, s), 0)

            if nVisited[s] >= n:
                finished[s] = True
                nFinished += 1

        k += 1

    for v in nodes:
        lower_bounds[v] = S_un[v]/ (n-1)

    return lower_bounds


def updateBoundsBFSCut(v, graph, x):
    """
    计算将顶点 v 分隔开的割集大小的下界。

    参数：
        v: 起始顶点。
        graph: networkx DiGraph。节点必须具有 'r' 属性。
        Farn: 存储下界的字典（必须初始化）。
        Top: top k 节点的列表（必须初始化）。
        x: 一个阈值。

    返回：
        如果割集值超过 x，则返回 +∞；否则返回计算出的割集值，或者如果无变化则返回当前 Farn[v]。
    """
    
    nodes = list(graph.nodes())
    n = len(nodes)
    Q = [(0,v)] #优先级队列，用于跟踪 BFS 的 (距离, 节点)
    heapq.heapify(Q)
    visited = {v}
    d = 0
    S = 0
    y = graph.degree(v) - 1
    nd = 1

    while Q:
        dist, u = heapq.heappop(Q)
        if dist > d:
            d += 1
            #LCUT 计算
            
            LCUT = ((d+2)*(n-nd) + S - y )/(n-1)
        

            if LCUT >= x:
                return float('inf')
            y = 0  # 重置 y

        for w in graph.neighbors(u):
            if w not in visited:
                visited.add(w)
                dist_vw = nx.shortest_path_length(graph,v,w) #假设存在最短路径
                heapq.heappush(Q,(dist_vw,w))
                S += dist_vw  # 距离 d(v,w)
                y += graph.degree(w)
                nd += 1
            else:
                LCUT = LCUT + 1/(n-1)

    #最终计算
    LCUT_final = S / (n-1)

    
    return LCUT_final


def create_mention_graph_with_centrality(filepath, top_k=20):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds...")

    initial_lower_bounds = neighborhood_lower_bound(graph)
    
    k_lowest_farness = heapq.nsmallest(top_k, initial_lower_bounds.items(), key=lambda item: item[1])
    print(f"\n{top_k} nodes with lowest farness values:")
    for node, farness in k_lowest_farness:
        print(f"Node: {node}, Farness: {farness}")
    
    Farn = initial_lower_bounds.copy()
    Top = []
    Q = [(Farn[node], node) for node in graph.nodes()]
    heapq.heapify(Q)

    initial_threshold = float('inf')


    _, v = heapq.heappop(Q)
    
    print(f"Processing node {v}")

    threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
    refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
    print(f"Refined bound for node {v}: {refined_bound}")

    Farn[v] = refined_bound

    while Q:
        
        if len(Top) < top_k:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            print(f"Added node {v} to Top. Current Top: {Top}")
        elif refined_bound < Farn[Top[-1]]:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            Top = Top[:top_k]  # 保持Top的长度为top_k
            print(f"Added node {v} to Top and trimmed. Current Top: {Top}")
        else:
            print(f"Node {v} not added to Top as its bound is not better than current top")

        _, v = heapq.heappop(Q)
    
        print(f"Processing node {v}")

        threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
        refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
        print(f"Refined bound for node {v}: {refined_bound}")

        Farn[v] = refined_bound
        
        if len(Top) == top_k and Farn[v] >= Farn[Top[-1]]:
            print(Farn[Q[0][1]])
            print(Farn[Top[-1]])
            print("Remaining nodes in Q cannot improve Top. Stopping.")
            break

    print("\nFinal Top 10 nodes by refined lower centrality bound:")
    for i, node in enumerate(Top, 1):
        print(f"{i}. Node: {node}, Refined Lower Centrality: {Farn[node]:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return graph

if __name__ == "__main__":
    filepath = "medium.tsv"  # 替换为你文件的路径
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

Creating graph...
Undirected graph created with 2158 nodes and 5136 edges.
Calculating initial lower bounds...

20 nodes with lowest farness values:
Node: 575, Farness: 3.001390820584145
Node: 374, Farness: 3.008344923504868
Node: 851, Farness: 3.0139082058414464
Node: 222, Farness: 3.044506258692629
Node: 46, Farness: 3.097357440890125
Node: 588, Farness: 3.09874826147427
Node: 682, Farness: 3.2962447844228095
Node: 628, Farness: 3.305980528511822
Node: 529, Farness: 3.3310152990264257
Node: 245, Farness: 3.388038942976356
Node: 817, Farness: 3.4089012517385258
Node: 770, Farness: 3.4255910987482614
Node: 118, Farness: 3.49095966620306
Node: 116, Farness: 3.4937413073713492
Node: 717, Farness: 3.5076495132127956
Node: 51, Farness: 3.5424200278164117
Node: 22, Farness: 3.556328233657858
Node: 572, Farness: 3.603616133518776
Node: 270, Farness: 3.631432545201669
Node: 611, Farness: 3.6481223922114046
Processing node 575
Refined bound for node 575: 2.477051460361613
Added node 575 to Top

In [4]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def neighborhood_lower_bound(graph):
    nodes = list(graph.nodes())
    n = len(nodes)
    lower_bounds = {}

    k = 2
    max_iterations = 100000  # 设置最大迭代次数

    Y = {}
    S_un = {}
    nVisited = {}
    finished = {}

    for s in nodes:
        degree_s = graph.degree(s)
        Y[(1, s)] = degree_s
        S_un[s] = degree_s
        nVisited[s] = degree_s + 1
        finished[s] = False

    nFinished = 0

    while nFinished < n and k <= max_iterations:
        for s in nodes:
            if finished[s]:
                continue
            if k == 2:
                Y[(k, s)] = sum(Y.get((k-1, w), 0) for w in graph.neighbors(s)) - graph.degree(s)
            else:
                Y[(k, s)] = sum(Y.get((k-1, w), 0) for w in graph.neighbors(s)) - Y.get((k-2, s), 0) * (graph.degree(s) - 1)
        
        for s in nodes:
            if finished[s]:
                continue
            y_k_minus_2 = Y.get((k-2, s), 0)
            y_k_minus_1 = Y.get((k-1, s), 0)
            nVisited[s] += y_k_minus_1
            
            if nVisited[s] < n:
                S_un[s] += k * y_k_minus_1
            else:
                S_un[s] += k *(n-(nVisited[s] - y_k_minus_1))
                nFinished += 1
                finished[s] = True
            Y[(k-2, s)] = y_k_minus_1
            Y[(k-1, s)] = Y[(k, s)]
        
        k += 1

    for v in nodes:
        lower_bounds[v] = S_un[v] / (n - 1)

    return lower_bounds

def updateBoundsBFSCut(v, graph, x):
    """
    计算将顶点 v 分隔开的割集大小的下界。

    参数：
        v: 起始顶点。
        graph: networkx DiGraph。节点必须具有 'r' 属性。
        Farn: 存储下界的字典（必须初始化）。
        Top: top k 节点的列表（必须初始化）。
        x: 一个阈值。

    返回：
        如果割集值超过 x，则返回 +∞；否则返回计算出的割集值，或者如果无变化则返回当前 Farn[v]。
    """
    
    nodes = list(graph.nodes())
    n = len(nodes)
    Q = [(0,v)] #优先级队列，用于跟踪 BFS 的 (距离, 节点)
    heapq.heapify(Q)
    visited = {v}
    d = 0
    S = 0
    y = graph.degree(v) - 1
    nd = 1

    while Q:
        dist, u = heapq.heappop(Q)
        if dist > d:
            d += 1
            #LCUT 计算
            
            LCUT = ((d+2)*(n-nd) + S - y )/(n-1)
        

            if LCUT >= x:
                return float('inf')
            y = 0  # 重置 y

        for w in graph.neighbors(u):
            if w not in visited:
                visited.add(w)
                dist_vw = nx.shortest_path_length(graph,v,w) #假设存在最短路径
                heapq.heappush(Q,(dist_vw,w))
                S += dist_vw  # 距离 d(v,w)
                y += graph.degree(w)
                nd += 1
            else:
                LCUT = LCUT + 1/(n-1)

    #最终计算
    LCUT_final = S / (n-1)

    
    return LCUT_final


def create_mention_graph_with_centrality(filepath, top_k=10):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds...")

    initial_lower_bounds = neighborhood_lower_bound(graph)
    
    k_lowest_farness = heapq.nsmallest(top_k, initial_lower_bounds.items(), key=lambda item: item[1])
    print(f"\n{top_k} nodes with lowest farness values:")
    for node, farness in k_lowest_farness:
        print(f"Node: {node}, Farness: {farness}")
    
    Farn = initial_lower_bounds.copy()
    Top = []
    Q = [(Farn[node], node) for node in graph.nodes()]
    heapq.heapify(Q)

    initial_threshold = float('inf')


    _, v = heapq.heappop(Q)
    
    print(f"Processing node {v}")

    threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
    refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
    print(f"Refined bound for node {v}: {refined_bound}")

    Farn[v] = refined_bound

    while Q:
        
        if len(Top) < top_k:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            print(f"Added node {v} to Top. Current Top: {Top}")
        elif refined_bound < Farn[Top[-1]]:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            Top = Top[:top_k]  # 保持Top的长度为top_k
            print(f"Added node {v} to Top and trimmed. Current Top: {Top}")
        else:
            print(f"Node {v} not added to Top as its bound is not better than current top")

        _, v = heapq.heappop(Q)
    
        print(f"Processing node {v}")

        threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
        refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
        print(f"Refined bound for node {v}: {refined_bound}")

        Farn[v] = refined_bound
        
        if len(Top) == top_k and Farn[v] >= Farn[Top[-1]]:
            print(Farn[Q[0][1]])
            print(Farn[Top[-1]])
            print("Remaining nodes in Q cannot improve Top. Stopping.")
            break

    print("\nFinal Top 10 nodes by refined lower centrality bound:")
    for i, node in enumerate(Top, 1):
        print(f"{i}. Node: {node}, Refined Lower Centrality: {Farn[node]:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return graph

if __name__ == "__main__":
    filepath = "com-amazon.ungraph.tsv"  # 替换为你文件的路径
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

Creating graph...
Undirected graph created with 2158 nodes and 5136 edges.
Calculating initial lower bounds...

10 nodes with lowest farness values:
Node: 2, Farness: 2.289290681502086
Node: 548, Farness: 2.611961057023644
Node: 691, Farness: 2.634214186369958
Node: 50, Farness: 2.66759388038943
Node: 44, Farness: 2.6689847009735743
Node: 565, Farness: 2.674547983310153
Node: 150, Farness: 2.6926286509040334
Node: 47, Farness: 2.7037552155771905
Node: 613, Farness: 2.7357440890125173
Node: 54, Farness: 2.7482614742698193
Processing node 2
Refined bound for node 2: 2.1010662957811777
Added node 2 to Top. Current Top: [2]
Processing node 548
Refined bound for node 548: 2.247102457116365
Added node 548 to Top. Current Top: [2, 548]
Processing node 691
Refined bound for node 691: 2.246175243393602
Added node 691 to Top. Current Top: [2, 691, 548]
Processing node 50
Refined bound for node 50: 1.9573481687528975
Added node 50 to Top. Current Top: [50, 2, 691, 548]
Processing node 44
Refined 

In [1]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def calculate_closeness_centrality(graph, top_k=10):
    """
    计算图中节点的接近中心性,并返回 top k 个最高中心性的节点。

    参数:
    graph (networkx.Graph): 输入图
    top_k (int): 返回前 k 个最高中心性的节点

    返回:
    dict: 包含 top k 个节点及其接近中心性值的字典
    """
    # 计算每个节点的接近中心性
    closeness_centrality = nx.closeness_centrality(graph)

    # 获取 top k 个最高中心性的节点
    top_nodes = heapq.nlargest(top_k, closeness_centrality, key=closeness_centrality.get)
    top_centrality = {node: closeness_centrality[node] for node in top_nodes}

    return top_centrality

def create_mention_graph_with_centrality(filepath, top_k=10):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")

    print("Calculating top k nodes by closeness centrality...")
    top_closeness = calculate_closeness_centrality(graph, top_k)

    print("\nTop 10 nodes by closeness centrality:")
    for i, (node, centrality) in enumerate(top_closeness.items(), 1):
        print(f"{i}. Node: {node}, Closeness Centrality: {centrality:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return graph

if __name__ == "__main__":
    filepath = "com-youtube.ungraph.tsv"  # 替换为你文件的路径
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

Creating graph...
Undirected graph created with 17560 nodes and 22732 edges.
Calculating top k nodes by closeness centrality...

Top 10 nodes by closeness centrality:
1. Node: 363, Closeness Centrality: 0.4734543101
2. Node: 106, Closeness Centrality: 0.4641186266
3. Node: 18, Closeness Centrality: 0.4505542441
4. Node: 4, Closeness Centrality: 0.4431293375
5. Node: 104, Closeness Centrality: 0.4408596751
6. Node: 78, Closeness Centrality: 0.4394144144
7. Node: 210, Closeness Centrality: 0.4315523004
8. Node: 730, Closeness Centrality: 0.4310861239
9. Node: 404, Closeness Centrality: 0.4255386181
10. Node: 91, Closeness Centrality: 0.4192092823

Total time taken: 33.0521 seconds


In [39]:
import heapq
import csv
import re
import time
import cProfile
from collections import defaultdict
import networkx as nx
import itertools
import pandas as pd

def pagerank_lower_bound(graph):
    # 计算每个节点的 PageRank 值
    page_ranks = nx.pagerank(graph)
    
    # 将 PageRank 值的倒数作为下界
    lower_bounds = {node: 1 / page_ranks[node] for node in graph.nodes()}
    
    return lower_bounds

def updateBoundsBFSCut(v, graph, x):
    """
    计算将顶点 v 分隔开的割集大小的下界。

    参数：
        v: 起始顶点。
        graph: networkx DiGraph。节点必须具有 'r' 属性。
        Farn: 存储下界的字典（必须初始化）。
        Top: top k 节点的列表（必须初始化）。
        x: 一个阈值。

    返回：
        如果割集值超过 x，则返回 +∞；否则返回计算出的割集值，或者如果无变化则返回当前 Farn[v]。
    """
    
    nodes = list(graph.nodes())
    n = len(nodes)
    Q = [(0,v)] #优先级队列，用于跟踪 BFS 的 (距离, 节点)
    heapq.heapify(Q)
    visited = {v}
    d = 0
    S = 0
    y = graph.degree(v) - 1
    nd = 1

    while Q:
        dist, u = heapq.heappop(Q)
        if dist > d:
            d += 1
            #LCUT 计算
            
            LCUT = ((d+2)*(n-nd) + S - y )/(n-1)
        

            if LCUT >= x:
                return float('inf')
            y = 0  # 重置 y

        for w in graph.neighbors(u):
            if w not in visited:
                visited.add(w)
                dist_vw = nx.shortest_path_length(graph,v,w) #假设存在最短路径
                heapq.heappush(Q,(dist_vw,w))
                S += dist_vw  # 距离 d(v,w)
                y += graph.degree(w)
                nd += 1
            else:
                LCUT = LCUT + 1/(n-1)

    #最终计算
    LCUT_final = S / (n-1)

    
    return LCUT_final


def create_mention_graph_with_centrality(filepath, top_k=10):
    start_time = time.time()

    data = pd.read_csv(filepath, sep="\t", header=None)
    directed_graph = nx.DiGraph()
    directed_graph.add_edges_from(data.values.tolist())
    graph = directed_graph.to_undirected()

    print(f"Undirected graph created with {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges.")
    print("Calculating initial lower bounds using PageRank...")

    initial_lower_bounds = pagerank_lower_bound(graph)
    
    k_lowest_farness = heapq.nsmallest(top_k, initial_lower_bounds.items(), key=lambda item: item[1])
    print(f"\n{top_k} nodes with lowest farness values:")
    for node, farness in k_lowest_farness:
        print(f"Node: {node}, Farness: {farness}")
    
    Farn = initial_lower_bounds.copy()
    Top = []
    Q = [(Farn[node], node) for node in graph.nodes()]
    heapq.heapify(Q)

    initial_threshold = float('inf')


    _, v = heapq.heappop(Q)
    
    print(f"Processing node {v}")

    threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
    refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
    print(f"Refined bound for node {v}: {refined_bound}")

    Farn[v] = refined_bound

    while Q:
        
        if len(Top) < top_k:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            print(f"Added node {v} to Top. Current Top: {Top}")
        elif refined_bound < Farn[Top[-1]]:
            Top.append(v)
            Top.sort(key=lambda node: Farn[node])
            Top = Top[:top_k]  # 保持Top的长度为top_k
            print(f"Added node {v} to Top and trimmed. Current Top: {Top}")
        else:
            print(f"Node {v} not added to Top as its bound is not better than current top")

        _, v = heapq.heappop(Q)
    
        print(f"Processing node {v}")

        threshold = initial_threshold if len(Top) < top_k else Farn[Top[-1]]
        refined_bound = updateBoundsBFSCut(v, graph, threshold)
    
        print(f"Refined bound for node {v}: {refined_bound}")

        Farn[v] = refined_bound
        
        if len(Top) == top_k and Farn[v] >= Farn[Top[-1]]:
            print(Farn[v])
            print(Farn[Top[-1]])
            print("Remaining nodes in Q cannot improve Top. Stopping.")
            break

    print("\nFinal Top 10 nodes by refined lower centrality bound:")
    for i, node in enumerate(Top, 1):
        print(f"{i}. Node: {node}, Refined Lower Centrality: {Farn[node]:.10f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"\nTotal time taken: {total_time:.4f} seconds")

    return graph

if __name__ == "__main__":
    filepath = "com-youtube.ungraph.tsv"  # 替换为你文件的路径
    print("Creating graph...")
    graph_with_centrality = create_mention_graph_with_centrality(filepath)

Creating graph...
Undirected graph created with 17560 nodes and 22732 edges.
Calculating initial lower bounds using PageRank...

10 nodes with lowest farness values:
Node: 106, Farness: 16.483694625298675
Node: 4, Farness: 17.24106987697447
Node: 104, Farness: 18.269346717532212
Node: 22, Farness: 23.126325862788764
Node: 78, Farness: 40.40284292522227
Node: 34, Farness: 44.21001879512881
Node: 72, Farness: 55.64863839124198
Node: 47, Farness: 60.82988337035862
Node: 33, Farness: 67.33797845220953
Node: 91, Farness: 87.623296934277
Processing node 106
Refined bound for node 106: 2.1546215615923456
Added node 106 to Top. Current Top: [106]
Processing node 4
Refined bound for node 4: 2.2566774873284356
Added node 4 to Top. Current Top: [106, 4]
Processing node 104
Refined bound for node 104: 2.2682954610171424
Added node 104 to Top. Current Top: [106, 4, 104]
Processing node 22
Refined bound for node 22: 2.688194088501623
Added node 22 to Top. Current Top: [106, 4, 104, 22]
Processing no